## Import Dataset

In [ ]:
!gdown --folder https://drive.google.com/drive/folders/1dydbU9HlSIgGQBzYMLogDNI27uO6wga7?usp=drive_link

## Load & Clean the Data

In [ ]:
import pandas as pd
tmp_df = pd.read_excel("/content/Load-data/Data_สถานีชาร์จ/รายงานสรุป-Demand-รายวัน-สถานีชาร์จ-01-2024.xlsx")

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

# Define your cleaning function
def clean_header_and_drop_unuse_row(tmp_df):
    tmp_df.columns = tmp_df.iloc[0]
    tmp_df = tmp_df[1:]
    tmp_df = tmp_df.reset_index(drop=True)
    if 'Date' in tmp_df.columns:
        tmp_df = tmp_df[~pd.isna(tmp_df['Date'])]
    return tmp_df

# Helper function to process a single file
def process_file(file_info):
    file_path, rel_path = file_info

    try:
        tmp_df = pd.read_excel(file_path)
        cleaned_df = clean_header_and_drop_unuse_row(tmp_df)

        # Construct new CSV path
        output_path = os.path.join("/content/cleaned_data", rel_path).replace(".xlsx", ".csv")
        os.makedirs(os.path.dirname(output_path), exist_ok=True)

        # Save to CSV
        cleaned_df.to_csv(output_path, index=False)
        return f"✅ Processed: {file_path}"
    except Exception as e:
        return f"❌ Error with {file_path}: {str(e)}"

# Gather all .xlsx files with relative paths
xlsx_files = []
root_dir = "/content/Load-data"

for subdir, _, files in os.walk(root_dir):
    for file in files:
        if file.endswith(".xlsx"):
            full_path = os.path.join(subdir, file)
            rel_path = os.path.relpath(full_path, root_dir)
            xlsx_files.append((full_path, rel_path))

# Run in parallel using Pool
with Pool(cpu_count()) as pool:
    results = list(tqdm(pool.imap_unordered(process_file, xlsx_files), total=len(xlsx_files)))

# Optional: Print summary
for res in results:
    print(res)

## Preprocess

In [ ]:
import os
import pandas as pd
import re
from datetime import datetime
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

# Updated function
def preprocess_convert_datatype_with_date(tmp_df, filename):
    # Extract MM-YYYY from filename
    match = re.search(r"(\d{2})-(\d{4})", filename)
    if not match:
        raise ValueError(f"Cannot extract date from filename: {filename}")

    start_month = int(match.group(1))
    start_year = int(match.group(2))

    # Generate datetime range
    num_days = len(tmp_df)
    date_range = pd.date_range(start=datetime(start_year, start_month, 1), periods=num_days, freq='D')
    tmp_df['Date'] = date_range

    # Convert all other columns to numeric
    time_cols = [col for col in tmp_df.columns if col != "Date"]
    tmp_df[time_cols] = tmp_df[time_cols].apply(pd.to_numeric, errors='coerce')

    return tmp_df

# Wrapper for parallel processing
def process_csv_file(file_info):
    file_path, rel_path = file_info

    try:
        tmp_df = pd.read_csv(file_path)
        processed_df = preprocess_convert_datatype_with_date(tmp_df, os.path.basename(file_path))

        # Extract station name from relative path
        station_name = os.path.normpath(rel_path).split(os.sep)[0]
        processed_df.insert(0, 'station_name', station_name)  # Add as first column

        # Save to new folder
        output_path = os.path.join("/content/preprocessed_data", rel_path)
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        processed_df.to_csv(output_path, index=False)
        return f"✅ Processed: {file_path}"
    except Exception as e:
        return f"❌ Error with {file_path}: {str(e)}"

# Collect files
csv_files = []
root_dir = "/content/cleaned_data"

for subdir, _, files in os.walk(root_dir):
    for file in files:
        if file.endswith(".csv"):
            full_path = os.path.join(subdir, file)
            rel_path = os.path.relpath(full_path, root_dir)
            csv_files.append((full_path, rel_path))

# Run in parallel
with Pool(cpu_count()) as pool:
    results = list(tqdm(pool.imap_unordered(process_csv_file, csv_files), total=len(csv_files)))

# Optional: Print summary
for res in results:
    print(res)

## EDA

In [ ]:
!unzip "/content/Roboto Prompt.zip" -d "font"

Archive:  /content/Roboto Prompt.zip
replace font/Roboto/OFL.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace font/Roboto/README.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
# ─────────────────────────────────────────────
# 📌 Set Thai Font: Prompt-Regular.ttf
# ─────────────────────────────────────────────
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings("ignore")
# Load custom font
font_path = "/content/font/Prompt/Prompt-Regular.ttf"
custom_font = fm.FontProperties(fname=font_path)
plt.rcParams['font.family'] = custom_font.get_name()

# ─────────────────────────────────────────────
# 📁 EDA by Subfolder (station)
# ─────────────────────────────────────────────
import pandas as pd
import os
from glob import glob
from tqdm import tqdm

# Folder containing preprocessed data
preprocessed_root = "/content/preprocessed_data"

# Step 1: Group CSVs by station folder
folder_groups = {}
for csv_file in glob(os.path.join(preprocessed_root, '**', '*.csv'), recursive=True):
    station_folder = os.path.normpath(csv_file).split(os.sep)[-2]  # Get station name from parent folder
    folder_groups.setdefault(station_folder, []).append(csv_file)

# Step 2: EDA loop
for station, files in folder_groups.items():
    print(f"\n📊 EDA สำหรับสถานี: {station}")

    # Combine CSVs for this station
    dfs = [pd.read_csv(f) for f in files]
    station_df = pd.concat(dfs, ignore_index=True)

    # Show info
    print("✅ จำนวนแถวและคอลัมน์:", station_df.shape)
    print("📅 ช่วงวันที่:", station_df['Date'].min(), "→", station_df['Date'].max())
    print("🧾 คอลัมน์:", station_df.columns.tolist())

    # Missing value summary
    print("❗ ค่า Missing:\n", station_df.isna().sum())

    # Total usage calculation
    time_cols = [col for col in station_df.columns if col not in ['Date', 'station_name']]
    station_df['TotalUsage'] = station_df[time_cols].sum(axis=1)

    station_df['Date'] = pd.to_datetime(station_df['Date'])
    daily_usage = station_df.groupby('Date')['TotalUsage'].sum()

    # Plot daily usage
    plt.figure(figsize=(10, 4))
    plt.plot(daily_usage.index, daily_usage.values)
    plt.title(f"{station} - การใช้ไฟฟ้ารวมรายวัน", fontsize=14, fontproperties=custom_font)
    plt.xlabel("วันที่", fontproperties=custom_font)
    plt.ylabel("การใช้ไฟฟ้า (kW)", fontproperties=custom_font)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

### Decomposition

In [ ]:
# ─────────────────────────────────────────────
# 📌 Suppress Warnings
# ─────────────────────────────────────────────
import warnings
warnings.filterwarnings("ignore")

# ─────────────────────────────────────────────
# 📌 Set Thai Font: Prompt-Regular.ttf
# ─────────────────────────────────────────────
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.dates as mdates

font_path = "/content/font/Prompt/Prompt-Regular.ttf"
custom_font = fm.FontProperties(fname=font_path)
plt.rcParams['font.family'] = custom_font.get_name()

# ─────────────────────────────────────────────
# 📦 Imports
# ─────────────────────────────────────────────
import pandas as pd
import os
from glob import glob
from tqdm import tqdm
from statsmodels.tsa.seasonal import seasonal_decompose

# ─────────────────────────────────────────────
# 📁 Seasonal Decomposition by Station
# ─────────────────────────────────────────────
preprocessed_root = "/content/preprocessed_data"

# Step 1: Group CSVs by station folder
folder_groups = {}
for csv_file in glob(os.path.join(preprocessed_root, '**', '*.csv'), recursive=True):
    station_folder = os.path.normpath(csv_file).split(os.sep)[-2]
    folder_groups.setdefault(station_folder, []).append(csv_file)

# Step 2: EDA + Decomposition
for station, files in folder_groups.items():
    print(f"\n📊 EDA + Decomposition สำหรับสถานี: {station}")

    # Combine CSVs
    dfs = [pd.read_csv(f) for f in files]
    station_df = pd.concat(dfs, ignore_index=True)

    # Parse dates
    station_df['Date'] = pd.to_datetime(station_df['Date'])

    # Set 'Date' as index for easier time series operations
    station_df = station_df.set_index('Date').sort_index()

    # Calculate TotalUsage
    time_cols = [col for col in station_df.columns if col not in ['station_name']] # 'Date' is now index
    station_df['TotalUsage'] = station_df[time_cols].sum(axis=1)

    # Group daily usage and resample to daily frequency, summing usage for each day
    # This step will create NaNs for any missing days automatically
    daily_usage = station_df['TotalUsage'].resample('D').sum()

    # Handle NaNs: For decomposition, it's critical to have no NaNs.
    # ffill is acceptable for short gaps, but be aware of its limitations.
    # If you have long periods of missing data, consider other imputation methods
    # or acknowledge that decomposition might be less accurate.
    daily_usage = daily_usage.fillna(method='ffill')
    # An alternative for decomposition if ffill is not desired is to interpolate:
    # daily_usage = daily_usage.interpolate(method='time') # This might be better for some cases

    # Check for remaining NaNs after fillna (should be none)
    if daily_usage.isnull().any():
        print(f"Warning: NaNs still present in {station} daily_usage after ffill. Decomposition might fail or be inaccurate.")
        continue # Skip to the next station if decomposition won't work

    # Seasonal decomposition (daily series, assumes 7-day weekly seasonality)
    # Ensure the series is long enough for the period. For period=7, you need at least 2*7 = 14 data points.
    if len(daily_usage) < 2 * 7: # Minimum 2 full cycles for reliable decomposition
        print(f"Skipping decomposition for {station}: Not enough data points ({len(daily_usage)}) for period=7.")
        continue


    result = seasonal_decompose(daily_usage, model='additive', period=7)

    fig = result.plot()
    fig.set_size_inches(16, 10)  # Wider and taller

    # Title in Thai with font
    fig.suptitle(f'{station} - การแยกองค์ประกอบของข้อมูลเวลา', fontsize=18, fontproperties=custom_font, y=1.02)

    # Improve tick frequency and format
    for ax in fig.axes:
        ax.set_ylabel("", fontproperties=custom_font)
        ax.tick_params(axis='x', labelrotation=45)

        # Set major ticks to weekly intervals for better detail
        ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=1))  # show every week
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%d-%b'))     # format: 01-Jul

    plt.tight_layout()
    plt.show()

## Preprocess

In [ ]:
# After processing is done, concatenate all processed files
all_data = []

output_root = "/content/preprocessed_data"

for subdir, _, files in os.walk(output_root):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(subdir, file)
            try:
                df = pd.read_csv(file_path)
                all_data.append(df)
            except Exception as e:
                print(f"❌ Failed to read {file_path}: {e}")

# Concatenate all data
if all_data:
    all_data_df = pd.concat(all_data, ignore_index=True)
    all_data_df.to_csv("/content/all_data_df.csv", index=False)
    print("✅ All data concatenated and saved to /content/all_data_df.csv")
else:
    print("⚠️ No data was loaded for concatenation.")
import re

# Identify time columns (HH:MM format)
time_columns = [col for col in all_data_df.columns if re.match(r"^\d{1,2}:\d{2}$", str(col))]

# Melt the DataFrame to long format
long_df = all_data_df.melt(
    id_vars=['station_name', 'Date'],
    value_vars=time_columns,
    var_name='Time',
    value_name='Electricity(kW)'
)

# Combine 'Date' and 'Time' into full datetime
long_df['Date'] = pd.to_datetime(long_df['Date'].astype(str) + ' ' + long_df['Time'])

# Drop 'Time' column
long_df.drop(columns=['Time'], inplace=True)

# Sort by station_name first, then by Date
long_df.sort_values(by=['station_name', 'Date'], inplace=True)

# Save to CSV
long_df.to_csv('/content/all_data_timeseries.csv', index=False)
print("✅ Time series data saved and sorted by station_name > Date to /content/all_data_timeseries.csv")

✅ All data concatenated and saved to /content/all_data_df.csv
✅ Time series data saved and sorted by station_name > Date to /content/all_data_timeseries.csv


## Define Weight

In [ ]:
import pandas as pd

def build_station_weights(df: pd.DataFrame) -> pd.DataFrame:
    """
    Compute normalized reverse weights per station using record counts:
      - weight = max_count / count
      - so the station with the most records has weight == 1
      - stations with fewer records get weights > 1

    Returns a DataFrame with columns:
      station_name, normalized_reverse_weight
    """
    station_counts = df['station_name'].value_counts()

    # Normalize so max count has weight = 1
    max_count = station_counts.max()
    normalized_reverse_weights = max_count / station_counts

    # Convert to DataFrame for easier viewing
    station_weights_df = normalized_reverse_weights.reset_index()
    station_weights_df.columns = ['station_name', 'normalized_reverse_weight']
    return station_weights_df
long_df = long_df[long_df['station_name']!= 'Data_อาคารวิทยนิเวศน์']
station_weights_df =build_station_weights(long_df)

In [ ]:
station_weights_df

,station_name,normalized_reverse_weight
0,Data_สถานีชาร์จ,1.000000
1,Data_อาคารจามจุรี 9,1.000000
2,Data_อาคารจุลจักรพงษ์,1.002786
3,Data_อาคารบรมราชกุมารี,1.002786
4,Data_อาคารจามจุรี4,1.094225


## Experiment [Fill mising value]

In [ ]:
long_df.loc[long_df['Electricity(kW)'] < 0, 'Electricity(kW)'] = 0

## Split train,valid and test

In [ ]:
# Define ratios
train_ratio = 0.8
test_ratio = 0.2  # Optional, just for clarity (1 - train_ratio)

# Create empty lists to collect per-station splits
train_list = []
test_list = []

# Split per station
for station, station_df in long_df.groupby('station_name'):
    station_df = station_df.sort_values('Date')
    n = len(station_df)

    train_end = int(n * train_ratio)

    train_list.append(station_df.iloc[:train_end])
    test_list.append(station_df.iloc[train_end:])

# Combine all stations back into global sets
train_df = pd.concat(train_list).reset_index(drop=True)
test_df = pd.concat(test_list).reset_index(drop=True)

# Save to CSV
train_df.to_csv('/content/train_timeseries.csv', index=False)
test_df.to_csv('/content/test_timeseries.csv', index=False)

print("✅ Split completed:")
print(f"Train set size: {len(train_df)} rows")
print(f"Test set size: {len(test_df)} rows")

✅ Split completed:
Train set size: 135705 rows
Test set size: 33927 rows


## Modeling

In [ ]:
!pip install -U autogluon

## Experiment

In [ ]:
# === Random Forest Tabular + Weighted WAPE (AutoGluon) ===
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor

RANDOM_SEED = 42
LABEL = "Electricity(kW)"
DATA_TRAIN = "/content/train_timeseries.csv"
DATA_TEST  = "/content/test_timeseries.csv"

# 1) Load
train_df = pd.read_csv(DATA_TRAIN)
test_df  = pd.read_csv(DATA_TEST)

# 1.1) Clean non-finite label values (if label exists in df)
def clean_non_finite(df: pd.DataFrame, label_col: str) -> pd.DataFrame:
    if label_col in df.columns:
        return df[np.isfinite(df[label_col])].copy()
    return df.copy()

train_df = clean_non_finite(train_df, LABEL)
test_df  = clean_non_finite(test_df, LABEL)

# 1.2) Parse datetime
train_df["Date"] = pd.to_datetime(train_df["Date"])
test_df["Date"]  = pd.to_datetime(test_df["Date"])

# 2) Feature engineering (basic time features; keep it tabular)
def add_time_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.sort_values(["station_name", "Date"]).copy()
    df["hour"] = df["Date"].dt.hour
    df["dayofweek"] = df["Date"].dt.dayofweek
    df["month"] = df["Date"].dt.month
    return df

train_tab = add_time_features(train_df)
test_tab  = add_time_features(test_df)

# (No lags/rollings here, so dropna is optional; keep for safety)
train_tab = train_tab.dropna().reset_index(drop=True)
test_tab  = test_tab.dropna().reset_index(drop=True)

# 3) Station weights (normalized reverse weight: max_count / count)
def build_station_weights(df: pd.DataFrame) -> pd.Series:
    """
    Returns a Series: index=station_name, value=normalized_reverse_weight
      - weight = max_count / count  (most-sampled station → 1.0, fewer samples → >1.0)
    """
    counts = df["station_name"].value_counts()
    max_count = counts.max()
    weights = (max_count / counts).astype(float)
    weights.name = "normalized_reverse_weight"
    return weights

# Use TRAIN distribution to define weights
station_weights = build_station_weights(train_tab)  # pd.Series

# 4) Fit AutoGluon Tabular with ONLY Random Forest
features = [c for c in train_tab.columns if c not in ["Date", LABEL]]
predictor = TabularPredictor(
    label=LABEL,
    path="AutogluonTabularModels_RF_only",
    eval_metric="mean_squared_error",
    problem_type="regression"
).fit(
    train_data=train_tab[features + [LABEL]],
    presets=None,
    time_limit=3600,
    hyperparameters={
        # 'RF' = RandomForest in AutoGluon Tabular
        "RF": {
            "random_state": RANDOM_SEED
        }
    }
)

# 5) Predict on test
y_pred = predictor.predict(test_tab[features])

# 6) Evaluate Weighted WAPE
#    WAPE = sum_i w_i * sum_t |y - yhat|  /  sum_i w_i * sum_t |y|
# NOTE: assumes test set HAS the label column
if LABEL not in test_tab.columns:
    raise ValueError("Test set has no label column; cannot compute WAPE.")

def weighted_wape(y_true: pd.Series, y_pred: pd.Series, stations: pd.Series, station_w: pd.Series) -> float:
    # map station -> weight; missing stations default to 1.0
    w = stations.map(station_w).fillna(1.0).to_numpy(dtype=float)
    ae = (y_true - y_pred).abs().to_numpy(dtype=float)
    abs_y = y_true.abs().to_numpy(dtype=float)
    num = (w * ae).sum()
    den = (w * abs_y).sum()
    # If all actuals are zero, fall back to unweighted MAE to avoid div/0
    return num / den if den != 0 else ae.mean()

# Prepare aligned Series
y_true_ser = test_tab[LABEL].reset_index(drop=True)
y_pred_ser = y_pred.reset_index(drop=True)
station_ser = test_tab["station_name"].reset_index(drop=True)

wwape = weighted_wape(y_true_ser, y_pred_ser, station_ser, station_weights)

# Also print plain WAPE for reference (no weights)
def plain_wape(y_true: pd.Series, y_pred: pd.Series) -> float:
    num = (y_true - y_pred).abs().sum()
    den = y_true.abs().sum()
    return float(num / den) if den != 0 else float((y_true - y_pred).abs().mean())

wape_plain = plain_wape(y_true_ser, y_pred_ser)

print("\n=== EVAL (RandomForest only) ===")
print(f"Weighted WAPE: {wwape:.6f}")
print(f"Plain WAPE   : {wape_plain:.6f}")

# Optionally: also MSE/MAE quick check
mse = np.mean((y_true_ser - y_pred_ser) ** 2)
mae = np.mean((y_true_ser - y_pred_ser).abs())
print(f"MAE: {mae:.6f}")
print(f"MSE: {mse:.6f}")

# If you still want a leaderboard (single model), you can pass the test set
# but AutoGluon's leaderboard won't know about the custom weighted metric:
# lb = predictor.leaderboard(test_tab[features + [LABEL]], silent=True)
# print(lb)


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       10.04 GB / 12.67 GB (79.2%)
Disk Space Avail:   66.46 GB / 112.64 GB (59.0%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' on datasets <30000 samples by using new models meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, and TabM. Absolute best accuracy. Requires a GPU. Recommended 64 GB CPU memory and 32+ GB GPU memory.
	presets='best'    : Maximize accuracy. Recommended for most users. Use in 


=== EVAL (RandomForest only) ===
Weighted WAPE: 0.367256
Plain WAPE   : 0.367232
MAE: 38.165514
MSE: 5927.953031
